<a href="https://colab.research.google.com/github/choeuneheol/python-practice/blob/master/0826moviereview.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets
!pip install konlpy
!pip install tqdm
!pip install git+https://github.com/haven-jeon/PyKoSpacing
!pip install git+https://github.com/ssut/py-hanspell.git
# datasets (huggingface 자연어처리 dataset)
# konlpy (한글 자연어처리 Tool)
# tqdm (반복문의 Progress Bar)
# PyKoSpacing (한글 띄어쓰기 보정 Tool)
# Py-Hanspell(네이버 하늘 맞춤법 검사기 기반 한글 맞춤법 보정 Tool)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 365 kB 5.1 MB/s 
     |████████████████████████████████| 212 kB 48.1 MB/s 
     |████████████████████████████████| 115 kB 42.9 MB/s 
     |████████████████████████████████| 120 kB 33.2 MB/s 
     |████████████████████████████████| 127 kB 40.2 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.4 MB 1.3 MB/s 
     |████████████████████████████████| 453 kB 55.3 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/haven-jeon/

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/ssut/py-hanspell.git to /tmp/pip-req-build-dv0zkfxc
  Running command git clone -q https://github.com/ssut/py-hanspell.git /tmp/pip-req-build-dv0zkfxc
  Created wheel for py-hanspell: filename=py_hanspell-1.1-py3-none-any.whl size=4868 sha256=a53d353f6a1b1be62f59b1486ad028141dc5aef7b96756ae88b0b222899ad5d6
  Stored in directory: /tmp/pip-ephem-wheel-cache-lz37jg6_/wheels/ab/f5/7b/d4124bb329c905301baed80e2ae45aa14e824f62ebc3ec2cc4
Successfully built py-hanspell


# 2-0. 데이터셋 준비
Naver Sentiment Movie Corpus(nsmc)
영화 리뷰 별점을 기준으로 긍정(1), 부정(0)으로 구분한 데이터셋

In [2]:
from datasets import load_dataset

#nsmc : 데이터셋 다운로드
dataset = load_dataset("nsmc")

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/150000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset nsmc downloaded and prepared to /root/.cache/huggingface/datasets/nsmc/default/1.1.0/bfd4729bf1a67114e5267e6916b9e4807010aeb238e4a3c2b95fbfa3a014b5f3. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
# dataset의 구조
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['id', 'document', 'label'],
        num_rows: 150000
    })
    test: Dataset({
        features: ['id', 'document', 'label'],
        num_rows: 50000
    })
})


In [3]:
# 첫 5개의 문장을 추출
for i in range(5):
  print(dataset["train"][i])

nsmc5_sents = [dataset["train"][idx]["document"] for idx in range(5)]

{'id': '9976970', 'document': '아 더빙.. 진짜 짜증나네요 목소리', 'label': 0}
{'id': '3819312', 'document': '흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나', 'label': 1}
{'id': '10265843', 'document': '너무재밓었다그래서보는것을추천한다', 'label': 0}
{'id': '9045019', 'document': '교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정', 'label': 0}
{'id': '6483659', 'document': '사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다', 'label': 1}


In [4]:
from konlpy.tag import Okt
okt = Okt()

# Okt 형태소 분석기로 문장을 형태소 단위의 token으로 분리
# 속도가 빠르다
# 정제되지 않은 데이터에서 좋은 성능을 보인다

for sent_i in nsmc5_sents:
  words_i = [w for w, p in okt.pos(sent_i)]
  print(sent_i)
  print(words_i)
  print()

아 더빙.. 진짜 짜증나네요 목소리
['아', '더빙', '..', '진짜', '짜증나네요', '목소리']

흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나
['흠', '...', '포스터', '보고', '초딩', '영화', '줄', '....', '오버', '연기', '조차', '가볍지', '않구나']

너무재밓었다그래서보는것을추천한다
['너', '무재', '밓었', '다그', '래서', '보는것을', '추천', '한', '다']

교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정
['교도소', '이야기', '구먼', '..', '솔직히', '재미', '는', '없다', '..', '평점', '조정']

사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다
['사이', '몬페', '그', '의', '익살스런', '연기', '가', '돋보였던', '영화', '!', '스파이더맨', '에서', '늙어', '보이기만', '했던', '커스틴', '던스트', '가', '너무나도', '이뻐', '보였다']



In [6]:
from hanspell import spell_checker
# 스펠체커를 통해서 맞춤법 고치고
from pykospacing import Spacing
# 스페싱이란 함수를 써서 띄어쓰기를 고침

spacing = Spacing()
tokenized_nsmc5_corpus = []

#한글 전처리 패키지인 hanspell과 pykospacing으로 전처리한 경우
for sent_i in nsmc5_sents:
  new_sent_i = spell_checker.check(sent_i).checked
  new_sent_i = spacing(new_sent_i)
  word_i = [w for w, p in okt.pos(new_sent_i)]

  tokenized_nsmc5_corpus.append(words_i)
  print("Before: {0}".format(sent_i))
  print("After: {0}".format(sent_i))
  print(word_i)
  print()
 # 전처리 완료

Before: 아 더빙.. 진짜 짜증나네요 목소리
After: 아 더빙.. 진짜 짜증나네요 목소리
['아', '더빙', '..', '진짜', '짜증', '나네요', '목소리']

Before: 흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나
After: 흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나
['흠', '...', '포스터', '보고', '초등학생', '영화', '줄', '....', '오버', '연기', '조차', '가볍지', '않구나']

Before: 너무재밓었다그래서보는것을추천한다
After: 너무재밓었다그래서보는것을추천한다
['너무', '재밓었다', '그래서', '보는', '것', '을', '추천', '한다']

Before: 교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정
After: 교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정
['교도소', '이야기', '구먼', '..', '솔직히', '재미', '는', '없다', '..', '평점', '조정']

Before: 사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다
After: 사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다
['사이먼', '페그', '의', '익살스러운', '연기', '가', '돋보였던', '영화', '!', '스파이더맨', '에서', '늙어', '보이기만', '했던', '커스틴', '던스트', '가', '너무나도', '이', '뻐', '보였다']



# 2-1. Bag of Words (Bow)